In [ ]:
import os, sys
sys.path.append('/root/BundleFusion_Ubuntu_Pangolin/build')
import time

import pyrealsense2 as rs
import numpy as np
import cv2

In [ ]:
pipeline = rs.pipeline()
profile = pipeline.start()

align_to = rs.stream.color
align = rs.align(align_to)

In [ ]:
from PyBundleFusion import BundleFusion

app_config_file = "../realsense_app_config.txt"
bundle_config_file = "../realsense_bundle_config.txt"
dataset_root = "../dataset/office2"

In [ ]:
bundle = BundleFusion(app_config_file, bundle_config_file)

In [ ]:
n_frame_per_save = 3

for i in range(100000):
    # Get frameset of color and depth
    frames = pipeline.wait_for_frames()
    # frames.get_depth_frame() is a 640x360 depth image

    # Align the depth frame to color frame
    aligned_frames = align.process(frames)

    # Get aligned frames
    aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
    color_frame = aligned_frames.get_color_frame()

    # Validate that both frames are valid
    # if not aligned_depth_frame or not color_frame:
    #     continue

    depth = np.asanyarray(aligned_depth_frame.get_data())
    color = np.asanyarray(color_frame.get_data())

    depth = cv2.Mat(depth)
    color = cv2.Mat(np.stack([color[...,2],color[...,1],color[...,0]],axis=-1))
    
    if (i+1) % n_frame_per_save == 0:
        result = bundle.process(color, depth)
    
        if not result:
            print("Fail!")
            break